In [1]:
import json
import os
import pickle
import shutil
import sys
import time
from datetime import timedelta, datetime

import igraph as ig
import numpy as np
import pandas as pd
import xgboost as xgb
from pyspark.sql import functions as sf
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.storagelevel import StorageLevel
from sklearn.metrics import f1_score, recall_score
from sklearn.ensemble import IsolationForest

import settings as s

os.environ["EXT_DATA_TYPE_FOLDER"] = "ethereum"

from common import get_weights, MULTI_PROC_STAGING_LOCATION, delete_large_vars
from communities import get_communities_spark
from features import (
    generate_features_spark, generate_features_udf_wrapper, 
    SCHEMA_FEAT_UDF, FEATURE_TYPES
)

%load_ext autoreload
%autoreload 2

In [2]:
if (
    sys.version_info.major,
    sys.version_info.minor,
    sys.version_info.micro,
) != (3, 11, 8):
    raise EnvironmentError(
        "Only runs efficiently on Python 3.11.8 (Tested on: Conda 24.1.2 | Apple M3 Pro)"
    )

In [3]:
SPARK_CONF = [
    ("spark.driver.memory", "32g"),
    ("spark.worker.memory", "32g"),
    ("spark.driver.maxResultSize", "32g"),
    ("spark.driver.bindAddress", "127.0.0.1"),
    ("spark.sql.execution.arrow.pyspark.enabled", "true"),
    ("spark.network.timeout", "600s"),
]

shutil.rmtree("artifacts", ignore_errors=True)
spark = (
    SparkSession.builder.appName("testing")
    .config(conf=SparkConf().setAll(SPARK_CONF))
    .getOrCreate()
)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/12 15:28:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
start = time.time()

In [5]:
data = pd.read_parquet(s.INPUT_DATA_FILE)
# Only interested when "target" is phishing
phishing_nodes = set(data.loc[data["is_phishing"], "target"].unique())
print(len(phishing_nodes))

1164


In [6]:
TRAIN_PERC = 0.65
VALIDATION_PERC = 0.15
TEST_PERC = 0.2

KEEP_TOP_N = 100

assert(sum([TRAIN_PERC, VALIDATION_PERC, TEST_PERC]) == 1)

In [7]:
%%time

source_firsts = data.groupby("source").agg(first_trx=("timestamp", "min"))
target_firsts = data.groupby("target").agg(first_trx=("timestamp", "min"))
active_since = source_firsts.join(target_firsts, lsuffix="_left", how="outer").fillna(datetime.now())
active_since.loc[:, "active_since"] = active_since.apply(lambda x: min([x["first_trx_left"], x["first_trx"]]), axis=1)
active_since = active_since.loc[:, ["active_since"]]
active_since.sort_values("active_since", inplace=True)

number_of_train_accounts = int(np.floor(active_since.shape[0] * TRAIN_PERC))
number_of_validation_accounts = int(np.floor(active_since.shape[0] * VALIDATION_PERC))
train_accounts = set(active_since.head(number_of_train_accounts).index.tolist())
assert len(train_accounts) == number_of_train_accounts
remaining = active_since.loc[~active_since.index.isin(train_accounts), :].sort_values("active_since")
validation_accounts = set(remaining.head(number_of_validation_accounts).index.tolist())
assert len(validation_accounts) == number_of_validation_accounts
test_accounts = set(active_since.index) - train_accounts - validation_accounts
print(f"{len(train_accounts):,} | {len(validation_accounts):,} | {len(test_accounts):,}")
assert sorted(train_accounts | validation_accounts | test_accounts) == sorted(active_since.index)

1,932,767 | 446,023 | 594,699
CPU times: user 24 s, sys: 282 ms, total: 24.3 s
Wall time: 24.3 s


In [8]:
data_orig = pd.read_parquet(s.INPUT_DATA_ORIG_FILE)

train = data_orig.loc[data_orig["source"].isin(train_accounts) & data_orig["target"].isin(train_accounts), :]
validation = data_orig.loc[data_orig["source"].isin(validation_accounts) & data_orig["target"].isin(validation_accounts), :]
test = data_orig.loc[data_orig["source"].isin(test_accounts) & data_orig["target"].isin(test_accounts), :]
print(
    round(train.shape[0] / data_orig.shape[0], 2), 
    round(validation.shape[0] / data_orig.shape[0], 2), 
    round(test.shape[0] / data_orig.shape[0], 2)
)

assert set(train.index).intersection(validation.index) == set()
assert set(validation.index).intersection(test.index) == set()
assert set(train.index).intersection(test.index) == set()

0.74 0.05 0.03


In [9]:
location_main = "features"
# shutil.rmtree(location_main, ignore_errors=True)

location_flow_dispense = f"{location_main}{os.sep}flow_dispense.parquet"
location_flow_passthrough = f"{location_main}{os.sep}flow_passthrough.parquet"
location_flow_sink = f"{location_main}{os.sep}flow_sink.parquet"

location_comm_as_source_features = f"{location_main}{os.sep}comm_as_source_features.parquet"
location_comm_as_target_features = f"{location_main}{os.sep}comm_as_target_features.parquet"
location_comm_as_passthrough_features = f"{location_main}{os.sep}comm_as_passthrough_features.parquet"
location_comm_as_passthrough_features_reverse = f"{location_main}{os.sep}comm_as_passthrough_features_reverse.parquet"

location_features_node_level = f"{location_main}{os.sep}features_node_level.parquet"
location_features_edges = f"{location_main}{os.sep}features_edges.parquet"

location_features_edges_train = f"{location_main}{os.sep}features_edges_train.parquet"
location_features_edges_valid = f"{location_main}{os.sep}features_edges_valid.parquet"
location_features_edges_test = f"{location_main}{os.sep}features_edges_test.parquet"

location_train_trx_features = f"{location_main}{os.sep}train_trx_features.parquet"
location_valid_trx_features = f"{location_main}{os.sep}valid_trx_features.parquet"
location_test_trx_features = f"{location_main}{os.sep}test_trx_features.parquet"

location_train_features = f"{location_main}{os.sep}train_features.parquet"
location_valid_features = f"{location_main}{os.sep}valid_features.parquet"
location_test_features = f"{location_main}{os.sep}test_features.parquet"

try:
    os.makedirs(location_main)
except FileExistsError:
    pass

In [10]:
data = spark.createDataFrame(data)

In [11]:
# Later on, we will reset the variables (to free up memory), while still keeping these intact
to_keep = %who_ls
to_keep = list(to_keep)

In [12]:
%%time

edges_totals = data.select("source", "target", "amount_usd").groupby(
    ["source", "target"]
).agg(sf.count("amount_usd").alias("amount_usd")).toPandas()
edges_totals = edges_totals.sort_values("amount_usd", ascending=False).reset_index(drop=True)
left_edges = spark.createDataFrame(edges_totals.groupby("target").head(KEEP_TOP_N).loc[:, ["source", "target"]])
right_edges = spark.createDataFrame(edges_totals.groupby("source").head(KEEP_TOP_N).loc[:, ["source", "target"]])

columns = ["source", "target", "timestamp", "amount_usd"]

left = left_edges.select(sf.col("source").alias("src"), sf.col("target").alias("tgt")).join(
    data.select(*columns),
    on=(sf.col("src") == sf.col("source")) & (sf.col("tgt") == sf.col("target")),
    how="left"
).drop("src", "tgt").persist(StorageLevel.DISK_ONLY)
select = []
for column in left.columns:
    select.append(sf.col(column).alias(f"left_{column}"))
left = left.select(*select)
right = right_edges.select(sf.col("source").alias("src"), sf.col("target").alias("tgt")).join(
    data.select(*columns),
    on=(sf.col("src") == sf.col("source")) & (sf.col("tgt") == sf.col("target")),
    how="left"
).drop("src", "tgt").persist(StorageLevel.DISK_ONLY)

print(left.count(), right.count())

25/07/12 15:30:55 WARN TaskSetManager: Stage 4 contains a task of very large size (9208 KiB). The maximum recommended task size is 1000 KiB.
[Stage 19:===================================================>  (192 + 8) / 200]

6890555 10494769
CPU times: user 5.41 s, sys: 465 ms, total: 5.88 s
Wall time: 4min 5s


In [13]:
%%time

flows_temporal = left.join(
    right,
    (left["left_target"] == right["source"]) &
    (left["left_timestamp"] <= right["timestamp"]),
    how="inner"
).groupby(["left_source", "left_target", "source", "target"]).agg(
    sf.sum("left_amount_usd").alias("left_amount_usd"),
    sf.sum("amount_usd").alias("amount_usd"),
).drop("left_target").select(
    sf.col("left_source").alias("dispense"),
    sf.col("source").alias("passthrough"),
    sf.col("target").alias("sink"),
    sf.least("left_amount_usd", "amount_usd").alias("amount_usd"),
)

aggregate = [
    sf.sum("amount_usd").alias("amount_sum"),
    sf.mean("amount_usd").alias("amount_mean"),
    sf.median("amount_usd").alias("amount_median"),
    sf.max("amount_usd").alias("amount_max"),
    sf.stddev("amount_usd").alias("amount_std"),
    sf.countDistinct("dispense").alias("dispense_count"),
    sf.countDistinct("passthrough").alias("passthrough_count"),
    sf.countDistinct("sink").alias("sink_count"),
]
for flow_location, flow_type in [
    (location_flow_dispense, "dispense"), (location_flow_passthrough, "passthrough"), (location_flow_sink, "sink")
]:
    print(flow_type)
    flows_temporal_stats = flows_temporal.groupby(flow_type).agg(*aggregate).toPandas()
    flows_temporal_cyclic_stats = flows_temporal.where(
        (sf.col("dispense") == sf.col("sink"))
    ).groupby(flow_type).agg(*aggregate).toPandas()
    flows_temporal_stats = flows_temporal_stats.set_index(flow_type).join(
        flows_temporal_cyclic_stats.set_index(flow_type),
        how="left", rsuffix="_cycle"
    )
    flows_temporal_stats.index.name = "key"
    flows_temporal_stats.to_parquet(flow_location)
    del flows_temporal_stats
    del flows_temporal_cyclic_stats

left.unpersist()
right.unpersist()

del edges_totals
del left_edges
del right_edges

dispense


passthrough


sink


[Stage 189:======================================>                 (9 + 4) / 13]

CPU times: user 644 ms, sys: 186 ms, total: 829 ms
Wall time: 12min 34s


In [14]:
%%time

data_input = data.select("*")
nodes_source = set(data.select("source").distinct().toPandas()["source"])
nodes_target = set(data.select("target").distinct().toPandas()["target"])
nodes_passthrough = nodes_source.intersection(nodes_target)

%run generate_flow_features.ipynb

comm_as_source_features.to_parquet(location_comm_as_source_features)
comm_as_target_features.to_parquet(location_comm_as_target_features)
comm_as_passthrough_features.to_parquet(location_comm_as_passthrough_features)
comm_as_passthrough_features_reverse.to_parquet(location_comm_as_passthrough_features_reverse)

del comm_as_source_features
del comm_as_target_features
del comm_as_passthrough_features
del comm_as_passthrough_features_reverse


Processing comm_as_source

Processed hop #1 | 4,097,207 | 2,113,092
Processed hop #2 | 28,186,799 | 1,061,379
Processed hop #3 | 27,626,031 | 906,951
Processed hop #4 | 33,319,933 | 902,616

Processing comm_as_target

Processed hop #1 | 1,595,960 | 1,119,024
Processed hop #2 | 36,986,915 | 1,109,888
Processed hop #3 | 37,572,633 | 1,088,264
Processed hop #4 | 50,059,052 | 1,084,408

Processing comm_as_passthrough

Processed hop #1 | 1,214,666 | 258,627
Processed hop #2 | 5,402,153 | 182,193
Processed hop #3 | 5,494,028 | 160,174
Processed hop #4 | 6,453,218 | 157,969

Processing comm_as_passthrough_reverse

Processed hop #1 | 537,548 | 258,628
Processed hop #2 | 8,850,244 | 250,410
Processed hop #3 | 9,209,746 | 247,019
Processed hop #4 | 11,328,001 | 246,129


comm_as_source_features

CPU times: user 2min 21s, sys: 8.97 s, total: 2min 30s
Wall time: 2min 26s

comm_as_target_features

CPU times: user 2min 10s, sys: 8.29 s, total: 2min 19s
Wall time: 2min 13s

comm_as_passthrough_featu

In [15]:
%%time

data_agg_weights = get_weights(
    data.groupby(["source", "target"])
    .agg(
        sf.sum("amount_usd").alias("amount")
    ).toPandas()
)

data_agg_weights_rev = data_agg_weights.rename(
    columns={"target": "source", "source": "target"}
).loc[:, ["source", "target", "weight"]]
data_agg_weights_ud = pd.concat([data_agg_weights, data_agg_weights_rev], ignore_index=True)
data_agg_weights_ud = data_agg_weights_ud.groupby(["source", "target"]).agg(weight=("weight", "sum")).reset_index()

data_agg_weights_ud.sort_values("weight", ascending=False, inplace=True)
grouped_ud = data_agg_weights_ud.groupby("source").head(KEEP_TOP_N).reset_index(drop=True)
grouped_ud = grouped_ud.groupby("source").agg(targets=("target", set))

total = grouped_ud.index.nunique()
nodes_neighborhoods = {}
for index, (source, targets) in enumerate(grouped_ud.iterrows()):
    community_candidates = {source}
    for target in targets["targets"]:
        community_candidates |= (grouped_ud.loc[target, "targets"] | {target})
    nodes_neighborhoods[source] = set(community_candidates)
    if not (index % 250_000):
        print(index, total)

del data_agg_weights_rev
del data_agg_weights_ud
del grouped_ud

0 2973489
250000 2973489
500000 2973489
750000 2973489
1000000 2973489
1250000 2973489
1500000 2973489
1750000 2973489
2000000 2973489
2250000 2973489
2500000 2973489
2750000 2973489
CPU times: user 2min 27s, sys: 3.47 s, total: 2min 31s
Wall time: 3min 44s


In [16]:
%%time

print("Constructing communities")

graph = ig.Graph.DataFrame(data_agg_weights, use_vids=False, directed=True)
communities = get_communities_spark(nodes_neighborhoods, graph, os.cpu_count(), spark)

del graph
del data_agg_weights
del nodes_neighborhoods

Constructing communities


CPU times: user 1min 24s, sys: 28.4 s, total: 1min 52s
Wall time: 14min 19s


In [17]:
%%time

ts_min = data.select(sf.min("timestamp").alias("x")).collect()[0]["x"] - timedelta(minutes=1)
data_graph_agg = data.groupby(["source", "target"]).agg(
    sf.sum("num_transactions").alias("num_transactions"),
    sf.sum("amount_usd").alias("amount_usd"),
    sf.collect_list(sf.array((sf.col("timestamp") - ts_min).cast("long"), sf.col("amount_usd"))).alias("timestamps_amounts"),
)
data_graph_agg_sdf = data_graph_agg.persist(StorageLevel.DISK_ONLY)
print(data_graph_agg_sdf.count())
data_graph_agg = data_graph_agg_sdf.toPandas()

5355155
CPU times: user 2.28 s, sys: 568 ms, total: 2.85 s
Wall time: 1min 22s


In [18]:
%%time

print("Communities features creation")

graph = ig.Graph.DataFrame(data_graph_agg, use_vids=False, directed=True)
features = generate_features_spark(communities, graph, spark)
features.columns = [f"{s.G_COMM_PREFIX}{x}" if x != "key" else x for x in features.columns]

del graph
del communities
del data_graph_agg

Communities features creation


CPU times: user 33min 15s, sys: 26min 10s, total: 59min 26s
Wall time: 1h 20min 31s


In [19]:
%%time

print("1-hop-source features creation")

features_source = data_graph_agg_sdf.withColumn("key", sf.col("source")).groupby("key").applyInPandas(
    generate_features_udf_wrapper(False), schema=SCHEMA_FEAT_UDF
).toPandas()
features_source = pd.DataFrame(features_source["features"].apply(json.loads).tolist())
types = {k: v for k, v in FEATURE_TYPES.items() if k in features_source.columns}
features_source = features_source.astype(types)
features_source.columns = [f"{s.G_1HOP_PREFIX}{x}" if x != "key" else x for x in features_source.columns]

1-hop-source features creation


CPU times: user 6.34 s, sys: 650 ms, total: 6.99 s
Wall time: 11min 34s


In [20]:
%%time

print("1-hop-target features creation")

features_target = data_graph_agg_sdf.withColumn("key", sf.col("target")).groupby("key").applyInPandas(
    generate_features_udf_wrapper(False), schema=SCHEMA_FEAT_UDF
).toPandas()
features_target = pd.DataFrame(features_target["features"].apply(json.loads).tolist())
types = {k: v for k, v in FEATURE_TYPES.items() if k in features_target.columns}
features_target = features_target.astype(types)
features_target.columns = [f"{s.G_1HOP_PREFIX}{x}" if x != "key" else x for x in features_target.columns]

1-hop-target features creation


CPU times: user 3.61 s, sys: 341 ms, total: 3.95 s
Wall time: 6min 53s


In [21]:
%%time

all_features = features.set_index("key").join(
    features_source.set_index("key"), how="left", rsuffix=f"_1_hop_as_source"
)
all_features.index.name = "key"
all_features = all_features.reset_index()

all_features = all_features.set_index("key").join(
    features_target.set_index("key"), how="left", rsuffix=f"_1_hop_as_target"
)

all_features = all_features.join(
    pd.read_parquet(location_comm_as_source_features), how="left", rsuffix="_dispense"
).join(
    pd.read_parquet(location_comm_as_target_features), how="left", rsuffix="_sink"
).join(
    pd.read_parquet(location_comm_as_passthrough_features), how="left", rsuffix="_passthrough"
).join(
    pd.read_parquet(location_comm_as_passthrough_features_reverse), how="left", rsuffix="_passthrough_rev"
).join(
    pd.read_parquet(location_flow_dispense), how="left", rsuffix="_dispense"
).join(
    pd.read_parquet(location_flow_passthrough), how="left", rsuffix="_passthrough"
).join(
    pd.read_parquet(location_flow_sink), how="left", rsuffix="_sink"
)

all_features.to_parquet(location_features_node_level)
del all_features

CPU times: user 19.5 s, sys: 4.39 s, total: 23.9 s
Wall time: 22.6 s


In [22]:
all_features = pd.read_parquet(location_features_node_level)

In [23]:
anomalies = all_features.loc[:, []]
anomalies.loc[:, "anomaly_score"] = IsolationForest().fit(
    all_features.fillna(0)
).decision_function(all_features.fillna(0))
anomalies.loc[:, "anomaly_score"] += abs(anomalies.loc[:, "anomaly_score"].min())

In [24]:
%%time

print(f"Generating edge features")

to_select = ["source", "target", "timestamp", "num_transactions", "amount", "amount_usd", "is_zero_transaction"]

edges_features_input = data.select(to_select).groupby(
    ["source", "target"]
).agg(
    sf.sum("num_transactions").alias("num_transactions"), 
    sf.sum("amount").alias("amount"),
    sf.sum("amount_usd").alias("amount_usd"),
    sf.count(sf.when(sf.col("is_zero_transaction"), 1).otherwise(0)).alias("count_zero_transactions"),
    sf.count(sf.when(sf.col("is_zero_transaction"), 0).otherwise(1)).alias("count_non_zero_transactions"),
    (sf.unix_timestamp(sf.max("timestamp")) - sf.unix_timestamp(sf.min("timestamp"))).alias("related_for"),
).persist(StorageLevel.DISK_ONLY)
_ = edges_features_input.count()

edge_features = edges_features_input.toPandas()
edge_features.to_parquet(location_features_edges)
del edge_features

Generating edge features


CPU times: user 2.15 s, sys: 512 ms, total: 2.66 s
Wall time: 1min 19s


In [25]:
edge_features = pd.read_parquet(location_features_edges)

In [26]:
%%time

train_edges = train.loc[:, ["source", "target"]].drop_duplicates().set_index(
    ["source", "target"]
)
valid_edges = validation.loc[:, ["source", "target"]].drop_duplicates().set_index(
    ["source", "target"]
)
test_edges = test.loc[:, ["source", "target"]].drop_duplicates().set_index(
    ["source", "target"]
)

train_features = train_edges.join(
    edge_features.set_index(["source", "target"]), how="left"
).reset_index()
validation_features = valid_edges.join(
    edge_features.set_index(["source", "target"]), how="left"
).reset_index()
test_features = test_edges.join(
    edge_features.set_index(["source", "target"]), how="left"
).reset_index()

CPU times: user 18 s, sys: 755 ms, total: 18.7 s
Wall time: 19.4 s


In [27]:
def save_edge_features(features_in, location):
    features_in = features_in.set_index("target").join(
        anomalies, how="left"
    ).reset_index().set_index("source").join(
        anomalies, how="left", rsuffix="_source"
    ).reset_index().set_index("target").join(
        all_features, how="left"
    ).reset_index().set_index("source").join(
        all_features, how="left", rsuffix="_source"
    ).reset_index()
    features_in.loc[:, "anom_scores_diff"] = features_in.loc[:, "anomaly_score"] - features_in.loc[:, "anomaly_score_source"]
    features_in.loc[:, "anom_scores_min"] = np.array(
        [
            features_in.loc[:, "anomaly_score"].values, 
            features_in.loc[:, "anomaly_score_source"].values
        ],
    ).min(axis=0)
    features_in.loc[:, "anom_scores_max"] = np.array(
        [
            features_in.loc[:, "anomaly_score"].values, 
            features_in.loc[:, "anomaly_score_source"].values
        ],
    ).max(axis=0)
    features_in.loc[:, "anom_scores_mean"] = np.array(
        [
            features_in.loc[:, "anomaly_score"].values, 
            features_in.loc[:, "anomaly_score_source"].values
        ],
    ).mean(axis=0)
    features_in.to_parquet(location)

In [28]:
%%time

save_edge_features(train_features, location_features_edges_train)

CPU times: user 39.1 s, sys: 15.9 s, total: 55 s
Wall time: 51.8 s


In [29]:
%%time

save_edge_features(validation_features, location_features_edges_valid)

CPU times: user 4.98 s, sys: 811 ms, total: 5.79 s
Wall time: 5.43 s


In [30]:
%%time

save_edge_features(test_features, location_features_edges_test)

CPU times: user 4.47 s, sys: 348 ms, total: 4.82 s
Wall time: 4.55 s


In [31]:
def save_trx_features(data_in, location):
    columns = ["source", "target", "amount", "amount_usd", "is_zero_transaction", "is_phishing"]
    trx_features = data_in.loc[:, columns]
    trx_features.to_parquet(location)
    del trx_features

In [32]:
%%time

save_trx_features(train, location_train_trx_features)
save_trx_features(validation, location_valid_trx_features)
save_trx_features(test, location_test_trx_features)

CPU times: user 1.83 s, sys: 147 ms, total: 1.97 s
Wall time: 2.12 s


In [33]:
# To free up memory for training

to_reset = %who_ls
to_reset = list(to_reset)
to_reset.remove("to_keep")
to_reset = set(to_reset) - set(to_keep)
for var_to_reset in list(to_reset):
    var_to_reset = f"^{var_to_reset}$"
    %reset_selective -f {var_to_reset}

delete_large_vars(globals(), locals())

Deleted `global` DataFrame: source_firsts
Deleted `global` DataFrame: target_firsts
Deleted `global` DataFrame: active_since
Deleted `global` large object: train_accounts
Deleted `global` DataFrame: remaining
Deleted `global` large object: validation_accounts
Deleted `global` large object: test_accounts
Deleted `global` DataFrame: data_orig
Deleted `global` DataFrame: train
Deleted `global` DataFrame: validation
Deleted `global` DataFrame: test


True

In [34]:
def combine_features(location_features_trx, location_features_edges, location_features):
    features_input = pd.read_parquet(location_features_edges)
    trx_features_input = pd.read_parquet(location_features_trx)
    features_input = trx_features_input.set_index(["source", "target"]).join(
        features_input.set_index(["source", "target"]),
        how="left",
        lsuffix="_trx"
    ).reset_index()
    features_input.to_parquet(location_features)
    del features_input

In [35]:
%%time

combine_features(location_train_trx_features, location_features_edges_train, location_train_features)
combine_features(location_valid_trx_features, location_features_edges_valid, location_valid_features)
combine_features(location_test_trx_features, location_features_edges_test, location_test_features)

CPU times: user 1min 46s, sys: 50.7 s, total: 2min 37s
Wall time: 2min 22s


In [36]:
shutil.rmtree(MULTI_PROC_STAGING_LOCATION, ignore_errors=True)

In [37]:
print((time.time() - start) // 60)
start = time.time()

167.0


In [38]:
train_features = pd.read_parquet(location_train_features)
validation_features = pd.read_parquet(location_valid_features)
test_features = pd.read_parquet(location_test_features)

In [39]:
%%time

missing_columns = (
    (set(train_features.columns).symmetric_difference(validation_features.columns)) |
    (set(train_features.columns).symmetric_difference(test_features.columns)) |
    (set(test_features.columns).symmetric_difference(validation_features.columns))
)
for column in missing_columns:
    if missing in train_features.columns:
        print(f"Deleting missing column from train: {column}")
        del train_features[column]
    if missing in validation_features.columns:
        print(f"Deleting missing column from validation: {column}")
        del validation_features[column]
    if missing in test_features.columns:
        print(f"Deleting missing column from test: {column}")
        del test_features[column]

train_features_labels = train_features.loc[:, ["is_phishing"]].copy(deep=True)
del train_features["is_phishing"]
del train_features["source"]
del train_features["target"]

validation_features_labels = validation_features.loc[:, ["is_phishing"]].copy(deep=True)
validation_features = validation_features.loc[:, train_features.columns]

test_features_labels = test_features.loc[:, ["is_phishing"]].copy(deep=True)
test_features = test_features.loc[:, train_features.columns]

CPU times: user 243 ms, sys: 337 ms, total: 580 ms
Wall time: 636 ms


In [64]:
def f1_eval(y, y_):
    return 1 - f1_score(y, np.round(y_))


def train_model(x, y, x_, y_, cv=False):
    if cv:
        model = xgb.XGBClassifier(
            early_stopping_rounds=10, scale_pos_weight=10,
            eval_metric=f1_eval, disable_default_eval_metric=True, 
            num_parallel_tree=10, max_depth=5,
            colsample_bytree=0.7, subsample=0.7, 
            n_estimators=100, enable_categorical=True,
        )
    else:
        model = xgb.XGBClassifier(
            early_stopping_rounds=10, scale_pos_weight=10,
            eval_metric=f1_eval, disable_default_eval_metric=True, 
            num_parallel_tree=10, max_depth=5,
            colsample_bytree=0.7, subsample=0.7, 
            n_estimators=100, enable_categorical=True,
        )
    model.fit(x, y, verbose=not cv, eval_set=[(x_, y_)])
    print(f"Best iteration: {model.best_iteration}\n")
    return model

In [68]:
%%time

model = train_model(
    train_features, train_features_labels["is_phishing"].values, 
    validation_features, validation_features_labels["is_phishing"].values,
)
y_test_predicted = model.predict(test_features)
f1_final = f1_score(test_features_labels["is_phishing"], y_test_predicted) * 100
print(
    round(f1_final, 2),
    round(recall_score(test_features_labels["is_phishing"], y_test_predicted) * 100, 2)
)
print()

[0]	validation_0-f1_eval:0.51290
[1]	validation_0-f1_eval:0.50900
[2]	validation_0-f1_eval:0.78553
[3]	validation_0-f1_eval:0.77150
[4]	validation_0-f1_eval:0.74224
[5]	validation_0-f1_eval:0.73637
[6]	validation_0-f1_eval:0.72823
[7]	validation_0-f1_eval:0.71816
[8]	validation_0-f1_eval:0.49157
[9]	validation_0-f1_eval:0.48297
[10]	validation_0-f1_eval:0.47102
[11]	validation_0-f1_eval:0.40227
[12]	validation_0-f1_eval:0.39372
[13]	validation_0-f1_eval:0.38162
[14]	validation_0-f1_eval:0.37469
[15]	validation_0-f1_eval:0.36915
[16]	validation_0-f1_eval:0.36755
[17]	validation_0-f1_eval:0.37303
[18]	validation_0-f1_eval:0.37884
[19]	validation_0-f1_eval:0.37471
[20]	validation_0-f1_eval:0.37253
[21]	validation_0-f1_eval:0.36773
[22]	validation_0-f1_eval:0.37561
[23]	validation_0-f1_eval:0.36775
[24]	validation_0-f1_eval:0.36655
[25]	validation_0-f1_eval:0.37700
[26]	validation_0-f1_eval:0.37631
[27]	validation_0-f1_eval:0.38303
[28]	validation_0-f1_eval:0.38395
[29]	validation_0-f1_eva

In [65]:
%%time

CV_FOLD_PERC = 0.8
N_FOLDS = 5

f1_scores = []
for fold in range(N_FOLDS):
    print("Fold", fold + 1)
    x_train = train_features.sample(frac=CV_FOLD_PERC)
    x_train_labels = x_train.loc[:, []].join(train_features_labels, how="left")
    x_validation = validation_features.sample(frac=CV_FOLD_PERC)
    x_validation_labels = x_validation.loc[:, []].join(validation_features_labels, how="left")
    model = train_model(
        x_train, x_train_labels["is_phishing"].values, 
        x_validation, x_validation_labels["is_phishing"].values,
        cv=True
    )
    y_test_predicted = model.predict(test_features)
    f1_cv = f1_score(test_features_labels["is_phishing"], y_test_predicted) * 100
    print(
        round(f1_cv, 2),
        round(recall_score(test_features_labels["is_phishing"], y_test_predicted) * 100, 2)
    )
    f1_scores.append(f1_cv)

Fold 1
Best iteration: 24

62.53 53.36
Fold 2
Best iteration: 22

61.81 53.47
Fold 3
Best iteration: 24

61.75 53.52
Fold 4
Best iteration: 24

61.52 53.01
Fold 5
Best iteration: 19

60.94 53.16
CPU times: user 4h 9min 46s, sys: 4min 43s, total: 4h 14min 30s
Wall time: 29min 25s


In [69]:
gfp_best = 51.49
gfp_std = 4.29

In [70]:
print(f"GFP best: {gfp_best} ± {gfp_std}")

GFP best: 51.49 ± 4.29


In [71]:
print(f"{round(f1_final, 2)} ±{round(np.std(f1_scores), 2)}")

62.54 ±0.51


In [72]:
uplift = round(((f1_final - gfp_best) / gfp_best) * 100, 2)
print(f"Uplift of {uplift}%")

Uplift of 21.46%


In [47]:
print((time.time() - start) // 60)

15.0
